In [1]:
import pickle
import pandas as pd
import geopandas
from shapely.geometry import Point
import matplotlib.pyplot as plt

# Below is our project

In [2]:
# read scraped CHA data
with open("data/CHA_rental_data.obj", "rb") as f:       
    d = pickle.load(f)
cha = pd.DataFrame.from_dict(data = d, orient = "index")

In [3]:
# clean CHA data
cols = ['Address','Monthly Rent','Property Type','Bath','Bed',
        'Voucher Necessary','Availability','Contact','URL','Lat','Long']
cha = cha[cols]
cha.Long = -1 * cha.Long

# correct error
cha.loc["4545145", "Long"] = -87.66593 
cha.loc["4545145", "Lat"] = 41.772175

In [4]:
cha.head()

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long
1288108,"1718 W 66th St 1, Chicago, IL 60636",800,Apt,1.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.774000,-87.667300
4012748,"6130 S Eberhart Ave 1, Chicago, IL 60637",1200,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783000,-87.613600
4017021,"4827 S Seeley Ave , Chicago, IL 60609",600,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.805800,-87.675600
4036551,"828 E 88th Pl 2, Chicago, IL 60619",875,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.734190,-87.603472
4036578,"4620 S Evans Ave 1, Chicago, IL 60653",1150,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.810646,-87.608165


In [5]:
cha.dtypes

Address               object
Monthly Rent           int64
Property Type         object
Bath                 float64
Bed                  float64
Voucher Necessary     object
Availability          object
Contact               object
URL                   object
Lat                  float64
Long                 float64
dtype: object

In [6]:
# convert to GeoDataFrame
cha['Coordinates'] = list(zip(cha.Long, cha.Lat))

In [7]:
cha['Coordinates'] = cha['Coordinates'].apply(Point)

In [8]:
gcha = geopandas.GeoDataFrame(cha, geometry='Coordinates')

In [9]:
gcha[203:204]

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates
4302747,"218 E 79th St 2, Chicago, IL 60619",1300,Apt,1.5,4.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.751146,-87.618888,POINT (-87.6188885 41.751146)


In [12]:
# read block group geojson
blocks_full = geopandas.read_file("data/block-groups.geojson")

In [13]:
blocks = blocks_full[['geometry', 'GEOID']]

In [14]:
blocks.head()

,geometry,GEOID
0,"(POLYGON ((-87.655192 41.983802, -87.655914999...",170310306041
1,(POLYGON ((-87.65111443516709 41.9840326124377...,170310306042
2,"(POLYGON ((-87.656031 41.987344, -87.655303 41...",170310306043
3,"(POLYGON ((-87.66006899999999 41.983622, -87.6...",170310307011
4,"(POLYGON ((-87.65454699999999 41.980646, -87.6...",170310307021


In [15]:
cha_with_geoid = geopandas.sjoin(gcha, blocks, how="left", op='intersects')

/Users/bhargaviganesh/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [16]:
cha_with_geoid.head()

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,index_right,GEOID
1288108,"1718 W 66th St 1, Chicago, IL 60636",800,Apt,1.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.774000,-87.667300,POINT (-87.6673 41.774),3028,170316712002
4012748,"6130 S Eberhart Ave 1, Chicago, IL 60637",1200,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783000,-87.613600,POINT (-87.61360000000001 41.783),3375,170314206001
4017021,"4827 S Seeley Ave , Chicago, IL 60609",600,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.805800,-87.675600,POINT (-87.6756 41.8058),557,170316115001
4036551,"828 E 88th Pl 2, Chicago, IL 60619",875,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.734190,-87.603472,POINT (-87.603472 41.73419),2756,170314408001
4036578,"4620 S Evans Ave 1, Chicago, IL 60653",1150,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.810646,-87.608165,POINT (-87.608165 41.810646),1178,170318436002


In [17]:
index_to_geoid = cha_with_geoid["GEOID"]

In [18]:
index_to_geoid.head()

1288108    170316712002
4012748    170314206001
4017021    170316115001
4036551    170314408001
4036578    170318436002
Name: GEOID, dtype: object

In [19]:
# write to csv
cha.to_csv("CHA_clean.csv")

In [20]:
cha_with_geoid.to_csv("CHA_with_geoid.csv")

In [21]:
shp_filepath = "data/ZillowNeighborhoods-IL.shp"

In [23]:
zillow_neighborhoods = geopandas.read_file(shp_filepath)

In [24]:
zillow_neighborhoods

,State,County,City,Name,RegionID,geometry
0,IL,Cook,Chicago,Wicker Park,7930,POLYGON ((-87.66743176018612 41.90457224446712...
1,IL,Cook,Chicago,Ravenswood,33597,POLYGON ((-87.66746751104743 41.96890188491437...
2,IL,Carroll,Lanark,Lake Carroll,35553,POLYGON ((-89.89474778399995 42.19632197400006...
3,IL,Cook,Chicago,Edgebrook,51847,POLYGON ((-87.76186400199992 42.00841336200006...
4,IL,Cook,Chicago,Clearing (W),137841,POLYGON ((-87.76188485799992 41.78500995800005...
5,IL,Cook,Chicago,Englewood,138261,POLYGON ((-87.62569999999994 41.79457000000009...
6,IL,Cook,Chicago,Pilsen,33420,POLYGON ((-87.64649745499992 41.85300464400007...
7,IL,Cook,Chicago,Dunning,138129,POLYGON ((-87.78618999999992 41.94558000000006...
8,IL,Cook,Chicago,McKinley Park,139408,POLYGON ((-87.66540663699993 41.82334798500005...
9,IL,Du Page,Wheaton,Northside,56394,POLYGON ((-88.09826626999995 41.87690400400004...


In [33]:
cha_geoid_zillow = geopandas.sjoin(cha_with_geoid, zillow_neighborhoods, how="left", op='intersects')

/Users/bhargaviganesh/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:44: UserWarning: CRS of frames being joined does not match!
  warn('CRS of frames being joined does not match!')


In [34]:
cha_geoid_zillow

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,GEOID,index_right,State,County,City,Name,RegionID
1288108,"1718 W 66th St 1, Chicago, IL 60636",800,Apt,1.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.774000,-87.667300,POINT (-87.6673 41.774),170316712002,82.0,IL,Cook,Chicago,West Englewood,269612
4012748,"6130 S Eberhart Ave 1, Chicago, IL 60637",1200,Apt,2.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.783000,-87.613600,POINT (-87.61360000000001 41.783),170314206001,202.0,IL,Cook,Chicago,West Woodlawn,403353
4017021,"4827 S Seeley Ave , Chicago, IL 60609",600,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.805800,-87.675600,POINT (-87.6756 41.8058),170316115001,143.0,IL,Cook,Chicago,Back of the Yards,403146
4036551,"828 E 88th Pl 2, Chicago, IL 60619",875,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.734190,-87.603472,POINT (-87.603472 41.73419),170314408001,10.0,IL,Cook,Chicago,Burnside,137634
4036578,"4620 S Evans Ave 1, Chicago, IL 60653",1150,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.810646,-87.608165,POINT (-87.608165 41.810646),170318436002,142.0,IL,Cook,Chicago,Bronzeville,403145
4062682,"8200 S Escanaba Ave 1r, Chicago, IL 60617",1000,Duplex,2.0,4.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.746300,-87.554300,POINT (-87.5543 41.7463),170314603024,24.0,IL,Cook,Chicago,South Chicago,140437
4062710,"211 N Laporte Ave 2, Chicago, IL 60644",950,Duplex,1.0,2.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.884900,-87.748800,POINT (-87.7488 41.8849),170312518005,176.0,IL,Cook,Chicago,South Austin,403120
4062715,"525 N Homan Ave , Chicago, IL 60624",1150,Apt,1.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.891100,-87.710900,POINT (-87.7109 41.8911),170318367001,54.0,IL,Cook,Chicago,East Garfield Park,269577
4062722,"1352 W 64th St 2, Chicago, IL 60636",1150,Apt,1.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.777990,-87.658923,POINT (-87.658923 41.77799),170318349002,5.0,IL,Cook,Chicago,Englewood,138261
4062766,"1352 W 64th St 2, Chicago, IL 60636",1150,Apt,1.0,3.0,Yes,Available Now,(773) 961-4367,http://chicagoha.gosection8.com/Section-8-hous...,41.777990,-87.658923,POINT (-87.658923 41.77799),170318349002,5.0,IL,Cook,Chicago,Englewood,138261


In [35]:
len(cha_geoid_zillow)

5276

In [36]:
len(cha)

5276

In [44]:
cha_geoid_zillow[cha_geoid_zillow["index_right"].isna()]

,Address,Monthly Rent,Property Type,Bath,Bed,Voucher Necessary,Availability,Contact,URL,Lat,Long,Coordinates,GEOID,index_right,State,County,City,Name,RegionID
4567448,"1050 W 79th St , Chicago, IL 60620",420,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.7472,-87.8484,POINT (-87.8484 41.7472),170318206031,NaN,NaN,NaN,NaN,NaN,NaN
4590237,"7549 S Union Ave , Chicago, IL 60620",980,Apt,1.0,2.0,Yes,Available Now,(708) 473-5464,http://chicagoha.gosection8.com/Section-8-hous...,42.2197,-88.5503,POINT (-88.55029999999999 42.2197),171118715002,NaN,NaN,NaN,NaN,NaN,NaN
4618006,"150 W Maple St , Chicago, IL 60610",890,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.5177,-87.6492,POINT (-87.64919999999999 41.5177),170318292001,NaN,NaN,NaN,NaN,NaN,NaN
4632177,"6742 S Champlain Ave , Chicago, IL 60637",1200,Apt,1.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.5402,-87.6008,POINT (-87.60080000000001 41.5402),170318287021,NaN,NaN,NaN,NaN,NaN,NaN
4640435,"6556 S Mozart St , Chicago, IL 60629",940,Apt,1.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.7351,-87.6936,POINT (-87.6936 41.7351),170318216002,NaN,NaN,NaN,NaN,NaN,NaN
4646604,"1791 W Howard St , Chicago, IL 60626",1525,Apt,2.0,2.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,42.0192,-87.6758,POINT (-87.6758 42.0192),170310102021,NaN,NaN,NaN,NaN,NaN,NaN
4729058,"11001 S Longwood Dr 1b, Chicago, IL 60643",800,Apt,1.0,1.0,Yes,Available Now,(708) 334-5387,http://chicagoha.gosection8.com/Section-8-hous...,41.7453,-87.7558,POINT (-87.75579999999999 41.7453),170318211021,NaN,NaN,NaN,NaN,NaN,NaN
4729219,"1151 S Austin Blvd 1 St Floor, Chicago, IL 60644",890,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.8665,-87.7741,POINT (-87.7741 41.8665),170318314002,NaN,NaN,NaN,NaN,NaN,NaN
4763759,"14132 S Hoxie Ave , Chicago, IL 60633",1150,Apt,1.0,3.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.6381,-87.5608,POINT (-87.5608 41.6381),170318257003,NaN,NaN,NaN,NaN,NaN,NaN
4796475,"11010 S Indiana Ave 1, Chicago, IL 60478",1150,Apt,1.0,1.0,Yes,Check Availability,None,http://chicagoha.gosection8.com/Section-8-hous...,41.6226,-87.6172,POINT (-87.6172 41.6226),170318272001,NaN,NaN,NaN,NaN,NaN,NaN
